# Comparison
Compare the population of two Anno Designer files. Residences where built has fewer residents than reference are yellow. If it is the other way round the residence is blue.


![Run all cells](imgs/run_all.png) ![Restart kernel](imgs/restart_kernel.png)

In [ ]:
import numpy as np

from IPython.display import clear_output
import codecs
from datetime import datetime
import pandas as pd
import re
import copy
import json
import math
import os
import pathlib
import requests
import threading
import ipywidgets as widgets
import IPython.display

one = np.array([1,1])



class Level :
    def __init__(self, index, level, template, radius, color) :
        self.index = index
        self.level = level
        self.template = template
        self.radius = radius
        self.color = color


A7PARAMS = {
        "languages": ["chinese", "english", "french", "german", "italian", "japanese", "korean", "polish", "russian",
                  "spanish", "taiwanese"],
    "levels": [
    Level(index = 0, level = 1, template="A7_residence_SkyScraper_5lvl1", radius = 4, color = {"A": 255, "R": 3, "G": 94, "B": 94}),
    Level(index = 1, level = 2, template="A7_residence_SkyScraper_5lvl2", radius = 4.25, color = {"A": 255, "R": 0, "G": 128, "B": 128}),
    Level(index = 2, level = 3, template="A7_residence_SkyScraper_5lvl3", radius = 5, color = {"A": 255, "R": 68, "G": 166, "B": 166}),
    Level(index = 3, level = 4, template="A7_residence_SkyScraper_5lvl4", radius = 6, color = {"A": 255, "R": 105, "G": 196, "B": 196}), 
    Level(index = 4, level = 5, template="A7_residence_SkyScraper_5lvl5", radius = 6.75, color = {"A": 255, "R": 161, "G": 234, "B": 234}),
]
}

def pos(obj) :
    return np.array([int(c) for c in obj['Position'].split(',')]) 

def size(obj) :
    return np.array([int(c) for c in obj['Size'].split(',')])

#simple image scaling to (nR x nC) size
def scale(im, nR, nC):
    nR0 = len(im)     # source number of self.rows 
    nC0 = len(im[0])  # source number of columns 
    return [[ im[int(nR0 * r / nR)][int(nC0 * c / nC)]  
             for c in range(nC)] for r in range(nR)]


        
class House :
    def __init__(self, obj, ID, tl = None) :
        self.obj = obj
        self.ID = ID
        
        self.tl = tl
        if self.tl is None :
            self.tl = pos(obj)
        self.size = np.array([3,3])
        self.center = self.tl + one
        self.level = 4
        
        for l in A7PARAMS["levels"] :
            if l.template == obj["Identifier"] :
                self.level = l.level
                break
                
        self.residents = None
        try:
            self.residents = int(obj["Label"])
        except:
            pass
        
    def __lt__(self,other):
        return self.center[1] < other.center[1] if self.center[0] == other.center[0] else self.center[0] < other.center[0]
        
    def dist(self, h) :
        return np.linalg.norm(self.center - h.center)
    
    def calculate_panorama(self, level = None) :
        if level is None :
            level = self.level
        
        support = level
            
        for n in self.neighbors :
            if A7PARAMS["levels"][level-1].radius < self.dist(n) :
                continue
                
            if n.level < level :
                support += 1
            else :
                support -= 1
                
        return max(0,min(5,support))      
           
    
                

class Layout :
    def __init__(self, ad_json) :
        r_max = math.ceil(A7PARAMS["levels"][-1].radius)
        
        self.json = ad_json
        
        self.tl = np.array([99999999, 999999999])
        self.br = np.array([-99999999, -999999999])
        one = np.array([1,1])
        for obj in ad_json['Objects'] :
            self.tl = np.minimum.reduce([self.tl, pos(obj)])
            self.br = np.maximum.reduce([self.br, pos(obj) + size(obj)])

        
        dim = self.br - self.tl
        self.rows = dim[1]
        self.cols = dim[0]
        self.area = np.empty(shape=dim, dtype=object)
       
        self.unique = dict()
        self.multiple = set()
        self.houses = []
        for obj in ad_json['Objects'] :
            p = pos(obj) - self.tl
            if self.is_street(obj) :
                continue
            
            if self.is_house(obj) :
                self.houses.append(House(obj, len(self.houses) + 1, p))
                h = self.houses[-1]  
                for x in range(h.tl[0], h.tl[0] + h.size[0]) :
                    for y in range(h.tl[1], h.tl[1] + h.size[1]) :
                        self.area[x][y] = h
            else:
                idx = obj.get("Identifier")
                if idx is None:
                    continue
                    
                if idx in self.multiple:
                    continue
                elif idx in self.unique:
                    self.multiple.add(idx)
                    del self.unique[idx]
                else:
                    self.unique[idx] = obj
            
       
    def is_house(self, obj) :
        if obj.get("Identifier") is None or obj.get("Template") is None:
            return False
        
        if obj.get("Identifier") == "Scholar_Residence" :
            return False
        
        return "residence" in obj["Identifier"].lower() or "skyscraper" in obj["Template"].lower()
    
    def is_street(self, obj):
        return obj.get("Road")
    
    def diff(self, other):
        offset = np.array([0,0])
        common_uniques = set(self.unique.keys()).intersection(set(other.unique.keys()))

        offsets=[]
        for key in common_uniques:
            if self.unique[key].get("Direction") == other.unique[key].get("Direction"):
                offsets.append(pos(other.unique[key]) - pos(self.unique[key]))
                               
        if len(offsets) >= 1:
            offset = np.median(np.array(offsets), axis=0).astype(int)
            
        print("Offset", offset)
                               
        ad_json = copy.deepcopy(self.json)
        ad_json["Modified"] = str(datetime.now().isoformat())
        for obj in ad_json["Objects"]:
            p = pos(obj) - self.tl
            h0 = self.area[p[0],p[1]]
            if h0 is None:
                continue
            
            p += offset
            h1 = other.area[p[0],p[1]]
            if h1 is None:
                obj["Color"] = {"A": 255, "R":0, "G":0, "B":0}
                continue
                
            r0 = h0.residents
            r1 = h1.residents

            if r0 == r1 :
                obj["Color"] = {"A": 255, "R": 255, "G": 255, "B": 255}
            elif r0 < r1 :
                g = max(0, int(255 - r1 + r0))
                obj["Color"] = {"A": 255, "R": 0, "G": g, "B": 255}
            else :
                g = max(0, int(255 + r1 - r0))
                obj["Color"] = {"A": 255, "R": 255, "G": g, "B": 0}
            
        return ad_json


    
class OptimizerGUI:
   
    def __init__(self):
        self.reference_file = None
        self.built_file = None

        self.vertical_margins = "0 0 1rem 0"
        self.horizontal_margins = "0 2rem 0 0"


        self.show()

    def compose_body(self):
        def callback(btn):
            self.select_file()

        def hide(elem):
            elem.layout.display = "none"

        self.btn_reference_file = widgets.FileUpload(accept='.ad', multiple=False, description="Reference")
        self.btn_reference_file.observe(callback)
        self.btn_built_file = widgets.FileUpload(accept='.ad', multiple=False, description="Built")
        self.btn_built_file.observe(callback)
        
        self.label_reference_file = widgets.Label()
        hide(self.label_reference_file)
        self.label_built_file = widgets.Label()
        hide(self.label_built_file)
        
        def callback_save(btn):
            self.save()
        
        self.btn_save = widgets.Button(description="Save Diff", disabled=True)
        self.btn_save.on_click(callback_save)

        w = widgets.HBox([
            self.btn_reference_file,
            self.label_reference_file,
            self.btn_built_file,
            self.label_built_file,
            self.btn_save
        ])
        
        for c in w.children:
            c.layout.margin = self.horizontal_margins
            
        return w


    def show(self):
        def hide(elem):
            elem.layout.display = 'none'

        self.body = self.compose_body()
        self.label_status = widgets.Label(value="")
        self.model = widgets.VBox([
            self.body,
            self.label_status
            ])

        for m in self.model.children:
            m.layout.margin = self.vertical_margins
            
        display(self.model)

    def set_status(self, msg):
        self.label_status.value = msg

    def select_file(self):
        def show(elem):
            elem.layout.display = None
            
        files = self.btn_reference_file.value
        if len(files) >= 1:      
        
            file = None
            if type(files) is dict:
                file = list(files.values())[0]
            else:
                file = files[0]


            self.reference_file = file
            self.label_reference_file.value = str(file["metadata"]["name"])
            show(self.label_reference_file)
        
        files = self.btn_built_file.value
        if len(files) >= 1:      
        
            file = None
            if type(files) is dict:
                file = list(files.values())[0]
            else:
                file = files[0]


            self.built_file = file
            self.label_built_file.value = str(file["metadata"]["name"])
            show(self.label_built_file)
            
        if self.reference_file is not None and self.built_file is not None:
            self.btn_save.disabled = False



    def save(self):
        def hide(elem):
            elem.layout.display = 'none'

        def show(elem):
            elem.layout.display = None      
        
        try:
            ad_json = json.loads(codecs.decode(self.reference_file["content"], encoding="utf-8"))
            ref_layout = Layout(ad_json)

            ad_json = json.loads(codecs.decode(self.built_file["content"], encoding="utf-8"))
            built_layout = Layout(ad_json)

            diff_layout = ref_layout.diff(built_layout)

            out_path = os.getcwd() + "\\" + pathlib.Path(self.reference_file["metadata"]["name"]).stem + "_diff.ad"

            with open(out_path, "w") as f :
                json.dump(diff_layout, f)
                self.set_status("Result written to: {}".format(out_path))

        except Exception as e:
            display(e)
            self.set_status(str(e))

clear_output(wait=True) 
ui = OptimizerGUI()